# Word2Vec Tutorial - The Skip-Gram Model

[원문 - Word2Vec Tutorial - The Skip-Gram Model](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/) - 발번역

이 튜토리얼은 Word2Vec의 skip gram 신경망 아키텍처에 대해 다룬다.
이 튜토리얼은 Word2Vec에 대한 일반적인 소개 및 추상적 통찰을 건너 뛰고 자세한 내용을 살펴 보는것이다.
특히 여기서는 skip gram 신경망 모델에 대해 얘기한다.

## 모델
skip-gram 신경망 모델은 사실 가장 기본적은 형태로 놀라울정도로 간단하다; 나는 설명을 채우기 시작하는 모든 작은 변환과 향상이라고 생각한다.

우리가 갈곳에 대한 높은 수준의 통찰력으로 시작하자. Word2Vec 은 기계 학습의 다른 곳에서 본 트릭을 사용한다. 우리는 특정 작업을 수행하기 위해 하나의 hidden 레이어로 간단한 신경망을 훈련할 것이다, 그러나 우리는 실제로 우리가 훈련 한 작업을 위해 신경망을 사용하지 않을것이다! 대신, 목표는 실제로 hidden 레이어의 가중치를 배우는 것이다. - 우리는 이 가중치가 실제로 우리가 배우려고 하는 "word vectors" 임을 알게 될것이다.


## 가짜작업
이제 우리는 수행 할 신경망을 구축할 이 "가짜" 작업에 대해 이야기 할 필요가 있다. 그런다음 나중에 이것이 간접적으로 우리가 실제로 사용하는 단어 벡터를 어떻게 제공하는지 다시 살펴보겠다.

우리는 신경망을 훈련시켜 다음과 같이 할것이다. 문장의 중간에 특정 단어가 주어지면(입력 단어), 이웃의 단어를 보고 무작위로 하나를 고른다. 네트워크는 우리 단어집안의 모든 단어가 우리가 선택한 "이웃 단어"가 될 확률을 말해줄것이다.

> '이웃된' 이라고 말하면 실제로 알고리즘에 대한 "창 크기" 매개변수가 있다. 일반적인 창 크기 (window size) 는 5이며, 타겟 단어 앞, 뒤 각가5개 씩 총 10개 단어를 의미한다

출력 확률은 입력 단어 근처의 각 어휘 단어를 얼마나 쉽게 찾을 수 있는지와 관련이 있다. 예를 들어 훈련된 네트워크에 입력단어 "Soviet"를 주면, 출력 확률은 "Union" and "Russia" 가 "수박"과 "캥거루"같은 관련없는 단어 보다 더 높을 것이다.

우린 훈련 문서에 있는 단어 쌍으로 신경망을 훈련할 것이다. 아래의 예는 "The quick brown fox jumps over the lazy dog." 이라는 문장에서 우리가 취할 훈련 샘플 (단어 쌍) 을 보여준다. 나는 예제를 위해 2의 작은 창 크기를 사용했다.(window size)
파란색 강조 표시된 단어가 입력단어다.
![training_data](http://mccormickml.com/assets/word2vec/training_data.png)

네트워크는 각 쌍이 표시되는 횟수로부터 통계를 학습한다. 예를들어 네트워크는 아마도 ("Soviet", "Sasquatch") 보다 ("soviet", "Union") 훈련 샘플을 더 많이 받을것이다. 훈련이 끝났을때, "Soviet" 를 입력으로 준다면, "Sasquatch" 보다 "Union" 혹은 "Russia"에 대한 확률이 훨씬 더 높을것이다.

## 모델 상세설명
이 모든 것이 어떻게 표현 되었습니까?

우선, 신경망에 텍스트 문자열로 단어를 공급할 수 없다는 것을 알기 때문에 단어를 네트워크에 표현할 방법이 필요하다. 이것을 하기위해, 우리는 먼저 훈련 문서에서 단어의 어휘를 구축한다.- 우리는 10,000 개의 고유 단어로 된 단어집이 있다고 가정해 보자.

우리는 "ant" 와 같은 입력단어를 one-hot 벡터로 나타낼것이다. 이 벡터는 10,000 요소를 가질 것이고(단어집안에 모든 단어당 하나),  "ant" 단어에 해당하는 위치에 1을 배치하고 다른 모든 위치에 "0" 을 배치한다.

네트워크의 출력은 우리 단어 사전의 모든 단어에 대해, 임의로 선택된 인접 단어가 그 어휘라는 확률을 포함하는 단일 벡터 (10,000 요소)다.

![skip_gram_net_arch](http://mccormickml.com/assets/word2vec/skip_gram_net_arch.png)

hidden 레이어 뉴런에는 활성화 함수가 없지만, 출력 뉴런에는 softmax 를 사용한다. 우리는 나중에 이것을 다시 볼것이다.

단어 쌍으로 이 네트워크를 훈련할 때, 입력은 입력 단어를 나타내는 one-hot 벡터고, 훈련 출력은 또한 출력 단어를 나타내는 one-hot 벡터다. 그러나 입력 단어로 훈련된 네트워크를 평가하면 출력 벡터는 실제로 확률 분포가 된다.(즉 단일 핫 벡터가 아닌 부동 소수점 값의 묶음)